In [2]:
import pandas as pd

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# @title Load Configuraiton 
import hbqaconfig
conf = hbqaconfig.setEnv('local')
# for k,v in conf.items(): print (k,":",v)

# Create Prompt for Questions Generation

In [65]:
df = pd.read_csv(conf['PE_FOLDER']+'03-Chunked_Book.csv')
print(df.shape)
df.head(3)

(1433, 7)


,Section_Id,Chunk_Id,Chunk,Chunk_Letters,Chunk_Words,Chunk_Approx_Tokens,Approx_Ques
0,Book01_002,4,"'Hear, O Sanjaya, all that happened thereupon ...",2742,505,672.0,3
1,Book01_003,8,"Section II\n""The Rishis said, 'O son of Suta, ...",5714,974,1295.0,6
2,Book01_003,9,"'Listen, O ye ascetics, to the outlines of the...",6353,974,1295.0,6


In [12]:
df = pd.read_csv(conf['PE_FOLDER']+'03-Chunked_Book_Incl_Small_Chunk.csv')
print(df.shape)
df.head(3)

(1697, 7)


,Section_Id,Chunk_Id,Chunk,Chunk_Letters,Chunk_Words,Chunk_Approx_Tokens,Approx_Ques
0,Book01_002,4,"'Hear, O Sanjaya, all that happened thereupon ...",2742,505,672.0,3
1,Book01_003,8,"Section II\n""The Rishis said, 'O son of Suta, ...",5714,974,1295.0,6
2,Book01_003,9,"'Listen, O ye ascetics, to the outlines of the...",6353,974,1295.0,6


In [13]:
df.index = df['Chunk_Id']
df

,Section_Id,Chunk_Id,Chunk,Chunk_Letters,Chunk_Words,Chunk_Approx_Tokens,Approx_Ques
Chunk_Id,,,,,,,
4,Book01_002,4,"'Hear, O Sanjaya, all that happened thereupon ...",2742,505,672.0,3
8,Book01_003,8,"Section II\n""The Rishis said, 'O son of Suta, ...",5714,974,1295.0,6
9,Book01_003,9,"'Listen, O ye ascetics, to the outlines of the...",6353,974,1295.0,6
15,Book01_003,15,"""The above are the contents of the Eighteen Pa...",2997,517,688.0,3
16,Book01_004,16,"Section III\n(Paushya Parva)\nSauti said, ""Jan...",5149,879,1169.0,6
...,...,...,...,...,...,...,...
2722,Book14_052,2722,When king Yudhishthira the just of Kuru's race...,1666,271,360.0,1
2767,Book14_092,2767,would do when seeing some injury done to him. ...,1948,339,451.0,1
2781,Book15_012,2781,"Section XII\n""Arjuna said, 'O Bhima, thou art ...",2146,369,491.0,1


In [15]:
def CreateQA_Prompt(df1, df_prompts, idx_of_files, context_id):
    for i in idx_of_files:
        Section_Id = df1.loc[i, "Section_Id"]
        Chunk_Id = df1.loc[i, "Chunk_Id"]
        Chunk_Words = df1.loc[i, "Chunk_Words"] 
        Chunk_Approx_Tokens = df1.loc[i, "Chunk_Approx_Tokens"]
        Chunk_Letters = df1.loc[i,"Chunk_Letters"]
        Questions = df1.loc[i,"Approx_Ques"]
        #question_to_generate = df1.loc[i,"Questions"]

        print ( '| Sectionid:', Section_Id,
                '| Chunk_Id:', Chunk_Id,
                "| Chunk Words:", Chunk_Words, 
                "| Chunk Approx Tokens:", Chunk_Approx_Tokens, 
                "| Chunk Letters:",Chunk_Letters,
                )
            
        text = df1.loc[i, "Chunk"]

        # prompt_ = f"""Write 1 question, corresponding answer, and their reference from the context based on the text below\n\nText: {chunk}\n\nQuestions: \n\nAnswer: \n\nReference:\n\n"""

        prompt_ = f"""Write {Questions} unique questions, corresponding answers and reference from the below context (do not repeat the context in your response)\n\nText: {text}\n\nFollow this template for your answer\n\nQuestion 1.\nAnswer 1.\nReference 1.\n\n"""

        new_row = Section_Id, Chunk_Id, Chunk_Letters, Chunk_Words, Chunk_Approx_Tokens, prompt_ ,""
        df_prompts.loc[i] = new_row

        context_id = context_id +1


    return context_id, df_prompts


In [16]:
# Initialize dataset for prompt 
df_prompts = pd.DataFrame(columns=['Section_Id','Chunk_Id','Chunk_Letters','Chunk_Words','Chunk_Approx_Tokens', 'Prompt','ChatGPT_Response'])
# Initialize context_id
chunk_id = 1 

# Generate Questions & Answers for Sections.
idx_of_files = df.index
chunk_id, df_prompts = CreateQA_Prompt(df, df_prompts, idx_of_files, chunk_id)
print(f"{chunk_id} Prompt Generated")


| Sectionid: Book01_002 | Chunk_Id: 4 | Chunk Words: 505 | Chunk Approx Tokens: 672.0 | Chunk Letters: 2742
| Sectionid: Book01_003 | Chunk_Id: 8 | Chunk Words: 974 | Chunk Approx Tokens: 1295.0 | Chunk Letters: 5714
| Sectionid: Book01_003 | Chunk_Id: 9 | Chunk Words: 974 | Chunk Approx Tokens: 1295.0 | Chunk Letters: 6353
| Sectionid: Book01_003 | Chunk_Id: 15 | Chunk Words: 517 | Chunk Approx Tokens: 688.0 | Chunk Letters: 2997
| Sectionid: Book01_004 | Chunk_Id: 16 | Chunk Words: 879 | Chunk Approx Tokens: 1169.0 | Chunk Letters: 5149
| Sectionid: Book01_006 | Chunk_Id: 22 | Chunk Words: 876 | Chunk Approx Tokens: 1165.0 | Chunk Letters: 5058
| Sectionid: Book01_007 | Chunk_Id: 23 | Chunk Words: 403 | Chunk Approx Tokens: 536.0 | Chunk Letters: 2267
| Sectionid: Book01_008 | Chunk_Id: 24 | Chunk Words: 896 | Chunk Approx Tokens: 1192.0 | Chunk Letters: 5092
| Sectionid: Book01_009 | Chunk_Id: 25 | Chunk Words: 642 | Chunk Approx Tokens: 854.0 | Chunk Letters: 3772
| Sectionid: Book

In [17]:
print(df_prompts.shape)
df_prompts.head(4)

(1697, 7)


,Section_Id,Chunk_Id,Chunk_Letters,Chunk_Words,Chunk_Approx_Tokens,Prompt,ChatGPT_Response
4,Book01_002,4,2742,505,672.0,"Write 3 unique questions, corresponding answer...",
8,Book01_003,8,5714,974,1295.0,"Write 6 unique questions, corresponding answer...",
9,Book01_003,9,6353,974,1295.0,"Write 6 unique questions, corresponding answer...",
15,Book01_003,15,2997,517,688.0,"Write 3 unique questions, corresponding answer...",


In [18]:
df_prompts.to_csv(conf['PE_FOLDER']+'04.1-Chunk_with_Prompts_Incl_Small_Chunk.csv',index=None)

In [19]:
df_prompts = pd.read_csv(conf['PE_FOLDER']+'04.1-Chunk_with_Prompts_Incl_Small_Chunk.csv')
df_prompts.head(33)

,Section_Id,Chunk_Id,Chunk_Letters,Chunk_Words,Chunk_Approx_Tokens,Prompt,ChatGPT_Response
0,Book01_002,4,2742,505,672.0,"Write 3 unique questions, corresponding answer...",NaN
1,Book01_003,8,5714,974,1295.0,"Write 6 unique questions, corresponding answer...",NaN
2,Book01_003,9,6353,974,1295.0,"Write 6 unique questions, corresponding answer...",NaN
3,Book01_003,15,2997,517,688.0,"Write 3 unique questions, corresponding answer...",NaN
4,Book01_004,16,5149,879,1169.0,"Write 6 unique questions, corresponding answer...",NaN
5,Book01_006,22,5058,876,1165.0,"Write 6 unique questions, corresponding answer...",NaN
6,Book01_007,23,2267,403,536.0,"Write 3 unique questions, corresponding answer...",NaN
7,Book01_008,24,5092,896,1192.0,"Write 6 unique questions, corresponding answer...",NaN
8,Book01_009,25,3772,642,854.0,"Write 4 unique questions, corresponding answer...",NaN
9,Book01_010,26,3630,641,853.0,"Write 4 unique questions, corresponding answer...",NaN
